In [10]:
from transformers import BertForMaskedLM, BertTokenizer

In [11]:
import os
import re

text_list=[]
for file in os.listdir("E:/Projects/IUI/Polish_poetry/")[:20]:
    with open("E:/Projects/IUI/Polish_poetry/"+file, "r", encoding="utf-8") as f:
        text = f.read()
        text = ' \n'.join([re.sub(r'[^\w\s]', '', re.sub    (r'-', ' ', i.lower().strip())) for i in text.splitlines() if i])
        if(text):
            text_list.append(text)

In [12]:
NUM = 1

In [14]:

import numpy as np
from RhymesScorer.score import score
from transformers import pipeline
import random
myscore = lambda x: sum(sum(score(x, type="advanced"),[]))


model = BertForMaskedLM.from_pretrained("./mlm_pretrained_model_from_scratch").to("cuda")
tokenizer = BertTokenizer.from_pretrained("./mlm_pretrained_model_from_scratch")

def mask_random_words(text, num_masks=1, num_masks_max=100):
    tokens = re.split(r'(\s+)', text)
    word_indices = [i for i, tok in enumerate(tokens) if tok.strip() != "" and not tok.isspace()]
    mask_positions = random.sample(word_indices[:100], num_masks)
    for pos in mask_positions:
        tokens[pos] = "[MASK]"
    test = "".join(tokens)
    return test

pipe = pipeline(task = "fill-mask", model=model, tokenizer=tokenizer)
tekst = mask_random_words(text_list[NUM],1)
# tekst = text_list[NUM].replace("wędrówce", "[MASK]")
print(myscore(text_list[NUM]), text_list[NUM])
print(myscore(tekst), tekst.replace("\n",""))

tekst2 = np.array(tekst.split())
idx = np.where(tekst2 == "[MASK]")
res = []
for preds in pipe(tekst):
    if '#' not in list(preds["token_str"]):
        res.append(preds["token_str"])
        break

tekst2[idx] = res
tekst3 = []
i = 0
for t in tekst.splitlines():
    for x in t.split():
        tekst3.append(tekst2[i])
        i += 1
    tekst3.append('\n')
tekst3 = " ".join(tekst3)
print(myscore(tekst3), tekst3.replace("\n"," ").replace("  ",""))

Device set to use cuda:0


80.0 błogosławione te jasne i proste 
serca wierzące z słodyczą gołębią 
które przetrwawszy ciężką losu chłostę 
szlachetnych pragnień w sobie nie wyziębią 
i chociaż niemi burza nieszczęść miota 
widzą wciąż piękno i dobro żywota 
kraków d 14 listopada 1887
80.0 błogosławione te jasne [MASK] proste serca wierzące z słodyczą gołębią które przetrwawszy ciężką losu chłostę szlachetnych pragnień w sobie nie wyziębią i chociaż niemi burza nieszczęść miota widzą wciąż piękno i dobro żywota kraków d 14 listopada 1887
80.0 błogosławione te jasne proste serca wierzące z słodyczą gołębią które przetrwawszy ciężką losu chłostę szlachetnych pragnień w sobie nie wyziębią i chociaż niemi burza nieszczęść miota widzą wciąż piękno i dobro żywota kraków d 14 listopada 1887
